In [14]:
import pandas as pd
filename = f'./data/conspiracy_submissions_redditapi.csv'
dataframe = pd.read_csv(filename)
conspiracy_file = open(f"./data/conspiracy_submissions_ukraine_words.txt", "r")
wordlist = conspiracy_file.read().split("\n")
wordlist = wordlist[:-1]
print('|'.join(wordlist))
dataframe = dataframe[dataframe["title"].str.contains('|'.join(wordlist))]
dataframe.shape

ukraine|russia|war|putin|russian|ukrainian|invasion|propaganda|russia ukraine|nuclear|nato|wef|conflict|military|ukraine russia|attack|nazi|nazis|zelensky|soldiers|war ukraine|fighting|russians|fight|ukraine war|sanctions|kyiv|nuclear war|kiev


(1214, 7)

In [3]:
dataframe.to_csv(f'./data/conspiracy_filtered.csv', header=True, index=False, columns=list(dataframe.axes[1]), encoding='utf-8')